In [1]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

Let's download all the data.

In [2]:
DATA_DIR = '/home/mlepekhin/data/domain'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_domain_lstm'
!mkdir {pathjoin(MODELS_DIR, MODEL_ID)}
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

mkdir: cannot create directory ‘/home/mlepekhin/models/allennlp_domain_lstm’: File exists


In [3]:
pd.read_csv(pathjoin(DATA_DIR, "en_test"))

,Unnamed: 0,text,domain
0,0,"connecting the Green MV bar ( on the right ) ,...",generated
1,1,"et player on the youngtor , when it does n ' t...",generated
2,2,Drivers cause ' life and death ' 999 delays in...,natural
3,3,Sometimes we get so lost in the excitement of ...,natural
4,4,Letter dated 7 October 2001 from the Chargé d ...,natural
...,...,...,...
215,215,The Free Software Definition We maintain this ...,natural
216,216,hon . Friend identified . Enforcement notices ...,generated
217,217,country . And then it ' s about leapfrog techn...,generated
218,218,Michael Yardy anchored Sussex 's 355 for eight...,natural


In [4]:
train_dataset_reader = build_domain_dataset_reader(lower=True)
val_dataset_reader = build_domain_dataset_reader(lower=True)

train_data, dev_data = read_data(
    pathjoin(DATA_DIR, "en_train"), 
    pathjoin(DATA_DIR, "en_test"),
    train_dataset_reader, 
    val_dataset_reader,
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

Reading data



Building the vocabulary


In [5]:
model = build_simple_lstm_model(vocab, num_layers=1, emb_size=128, hidden_size=128, bidirectional=False)

Building the model


In [6]:
if torch.cuda.is_available():
    cuda_device = 2
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

2


In [7]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [8]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    5,
    cuda_device=cuda_device,
    learning_rate=0.001,
)
print("Starting training")
trainer.train()
print("Finished training")

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Starting training



Finished training


In [8]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
!ls -lh "{CHECKPOINTS_DIR}"

total 167M
-rw-rw-r-- 1 mlepekhin mlepekhin 167M сен 15 19:46 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K сен 15 19:44 log


In [9]:
!gpustat

seven                Tue Sep 15 19:52:21 2020  440.33.01
[0] TITAN RTX        | 87'C,  94 % | 15395 / 24220 MB | sharuev(15383M)
[1] TITAN RTX        | 90'C,  94 % | 17513 / 24220 MB | sharuev(17501M)
[2] TITAN RTX        | 77'C,  95 % | 11375 / 24220 MB | sharuev(11363M)
[3] TITAN RTX        | 84'C,   4 % | 19435 / 24220 MB | mlepekhin(19423M)
